In [2]:
from PIL import Image
from glob import glob
from tqdm import tqdm
import numpy as np
import os
import pickle
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# load jpgs
def process_images(inpath, outpath, dim_tuple, extension, start=1):
    #open images
    for file in tqdm(glob(f'{inpath}*.{extension}')):
        with Image.open(file) as img:
            #rotate image
            rotated_images = rotate_images(img)
            
            # resize images
            resized = resizing(rotated_images, dim_tuple)
            
            #grayscale images
            gray_images = grayscale(resized)
            
            
            # save the images
            save_preprocessed_images(gray_images, outpath, extension, start)
            start += 3         
             
    return

In [ ]:
# rotate images
def rotate_images(image):
    '''
    rotate_images(image):
    This function rotates an image on it's center 7 times (45, 90, 135, 180, 225, 270, and mirror image)
    Input:
        One image file
    Returns:
        A list of images containing the original image and the rotated one
    '''
    rotated_images = []

    rotate180 = image.rotate(180)
    
    chirl_image = image.transpose(Image.FLIP_LEFT_RIGHT)
    
    rotated_images.extend([image, rotate180, chirl_image])  
    return rotated_images

In [ ]:
def resizing(images, dim_tuple):
    '''
    resizing(images, dim_tuple):
    This function resizes a list of images
    Input:
        List of images
        Tuple containing the desired hight and the width
    Returns:
        List of resized images        
    '''
    resized = [image.resize(dim_tuple) for image in images]
    return resized

In [ ]:
def grayscale(images):
    '''
    grayscale(images):
    This transforms RGB images to grayscale images
    Input:
        List of RBG images
    Returns:
        List of grayscale images
    '''
    gray_images = [image.convert(mode='L') for image in images]
    return gray_images

In [ ]:
# save images in a different path
def save_preprocessed_images(processed_images, outpath, extension, start):
    '''
    save_preprocessed_images(processed_images, outpath, extension, start)
    This function saves any type of image in a specific directory
    Input:
        processed_images: List of images
        outpath: Where you want the files to be saved
    Returns:
        Does not return a variable. Creates image files 
    '''
    [image.save(f'{outpath}\\image{i}.{extension}') for i, image in enumerate(processed_images, start)]
    return

In [ ]:
def pickle_train_test(inpath, train_path, test_path, extension, n, label):
    '''
    pickle_train_test(inpath, train_path, test_path, extension, n, label):
    This function creates two datasets from the contents of a specific directory
    Input:
        inpath: The path your files are at
        train_path: The path you want to save your train set
        test_path: The path you want to save your test set
        n: Train set size
        label: Label (int)
            
    '''
    images = []
    #open images
    for file in tqdm(glob(f'{inpath}*.{extension}')):
        with Image.open(file) as img:
            np_image = np.asarray(img) / 255
            images.append([np_image, label])
            if len(images) == n:
                with open(train_path, 'wb') as file:
                    pickle.dump(images, file)
                del images
                images = []
            
    with open(test_path, 'wb') as file:
        pickle.dump(images, file)
    del images
    return 

# Process Damselflies Images

In [ ]:
# create and save new images
inpath = r'E:\images2019\train_val2019\Damselflies\\'
extension = 'jpg'
outpath = r'E:\processed_damsel_images'
dim_tuple = (256, 256)

# process_images(inpath, outpath, dim_tuple, extension)

# Process Dragonflies Images

In [ ]:
inpath = r'E:\images2019\train_val2019\Dragonflies\\'
extension = 'jpg'
outpath = r'E:\processed_dragon_images'
dim_tuple = (256, 256)

# process_images(inpath, outpath, dim_tuple, extension)

# Train/Test Dragonfly Images

In [ ]:
inpath = r'E:\processed_dragon_images\\'
test_path = r'E:\processed_dragon_images\test\dragon_test.pkl'
train_path = r'E:\processed_dragon_images\train\dragon_train.pkl'
extension = 'jpg'

files = os.listdir(inpath) # dir is your directory path
n_files = len(files)
n_train = int(n_files * 0.8)
print(f'{n_files}, {n_train}')

# pickle_train_test(inpath, train_path, test_path, extension, n_train, 0)

# Train/Test Damselfly Images

In [ ]:
inpath = r'E:\processed_damsel_images\\'
test_path = r'E:\processed_damsel_images\test\damsel_test.pkl'
train_path = r'E:\processed_damsel_images\train\damsel_train.pkl'
extension = 'jpg'

files = os.listdir(inpath) # dir is your directory path
n_files = len(files)
n_train = int(n_files * 0.8)
print(f'{n_files}, {n_train}')

# pickle_train_test(inpath, train_path, test_path, extension, n_train, 1)

In [42]:
train_path = r'E:\processed_damsel_images\train\damsel_train.pkl'
with open(train_path, "rb") as train_file:
    damsel_train = pickle.load(train_file)

In [4]:
print(type(damsel_train))
print(damsel_train[0])
print(type(damsel_train[0]))
print(len(damsel_train))
print(len(damsel_train[0]))

<class 'list'>
[array([[0.34509804, 0.35294118, 0.39607843, ..., 0.33333333, 0.35294118,
        0.35294118],
       [0.33333333, 0.34117647, 0.38431373, ..., 0.3372549 , 0.35686275,
        0.35294118],
       [0.32941176, 0.3372549 , 0.38039216, ..., 0.34901961, 0.36078431,
        0.35686275],
       ...,
       [0.46666667, 0.45490196, 0.44313725, ..., 0.6627451 , 0.71764706,
        0.74509804],
       [0.4627451 , 0.45490196, 0.45098039, ..., 0.64705882, 0.67843137,
        0.68627451],
       [0.4627451 , 0.45490196, 0.45490196, ..., 0.64313725, 0.65882353,
        0.64705882]]), 1]
<class 'list'>
20310
2


In [ ]:
dragon = [np.array(cv2.imread(file)) for file in tqdm(glob(r'E:\processed_dragon_images\*.jpg'))]
print('Got dragonflies')

In [ ]:
image = plt.imshow(damsel_train[0][0], cmap='gray')

In [ ]:
image = cv2.imread(r'E:\processed_damsel_images\image1.jpg')
image

In [ ]:
import cv2
import numpy as np

#== Parameters =======================================================================
BLUR = 21
CANNY_THRESH_1 = 10
CANNY_THRESH_2 = 300
MASK_DILATE_ITER = 5
MASK_ERODE_ITER = 5
MASK_COLOR = (0.0,0.0,1.0) # In BGR format


#== Processing =======================================================================

#-- Read image -----------------------------------------------------------------------
img = cv2.imread(r'E:\processed_damsel_images\image1.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#-- Edge detection -------------------------------------------------------------------
edges = cv2.Canny(gray, CANNY_THRESH_1, CANNY_THRESH_2)
edges = cv2.dilate(edges, None)
edges = cv2.erode(edges, None)

#-- Find contours in edges, sort by area ---------------------------------------------
contour_info = []
# _, contours, B = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
# Previously, for a previous version of cv2, this line was: 
contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
# Thanks to notes from commenters, I've updated the code but left this note
for c in contours:
    contour_info.append((c,
                         cv2.isContourConvex(c),
                         cv2.contourArea(c)
                       ))
contour_info = sorted(contour_info, key=lambda c: c[2], reverse=True)
max_contour = contour_info[0]

#-- Create empty mask, draw filled polygon on it corresponding to largest contour ----
# Mask is black, polygon is white
mask = np.zeros(edges.shape)
cv2.fillConvexPoly(mask, max_contour[0], (255))

#-- Smooth mask, then blur it --------------------------------------------------------
mask = cv2.dilate(mask, None, iterations=MASK_DILATE_ITER)
mask = cv2.erode(mask, None, iterations=MASK_ERODE_ITER)
mask = cv2.GaussianBlur(mask, (BLUR, BLUR), 0)
mask_stack = np.dstack([mask]*3)    # Create 3-channel alpha mask

#-- Blend masked img into MASK_COLOR background --------------------------------------
mask_stack  = mask_stack.astype('float32') / 255.0          # Use float matrices, 
img         = img.astype('float32') / 255.0                 #  for easy blending

masked = (mask_stack * img) + ((1-mask_stack) * MASK_COLOR) # Blend
masked = (masked * 255).astype('uint8')                     # Convert back to 8-bit 

# split image into channels
c_red, c_green, c_blue = cv2.split(img)

# merge with mask got on one of a previous steps
img_a = cv2.merge((c_red, c_green, c_blue, mask.astype('float32') / 255.0))

# show on screen (optional in jupiter)

plt.imshow(img_a)
plt.show()

# cv2.imshow('img', masked)                                   # Display
# cv2.waitKey()

#cv2.imwrite('C:/Temp/person-masked.jpg', masked)           # Save

In [7]:
import tensorflow
import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from time import time
from sklearn.cluster import KMeans
from keras import callbacks
from keras.models import Model
from keras.optimizers import SGD
from keras.layers import Dense, Input
from keras.initializers import VarianceScaling
from keras.engine.topology import Layer, InputSpec
from tensorflow.keras import backend as k
from scipy.misc import imread
from sklearn.metrics import accuracy_score, normalized_mutual_info_score


Using TensorFlow backend.


In [48]:
len(damsel_train[0])

2

In [57]:
damsel_tensors[:17000]

array([[0.34509804, 0.35294118, 0.39607843, ..., 0.33333333, 0.35294118,
        0.35294118],
       [0.80392157, 0.82352941, 0.82745098, ..., 0.6745098 , 0.71764706,
        0.6627451 ],
       [0.54509804, 0.5254902 , 0.51764706, ..., 0.79215686, 0.78823529,
        0.78823529],
       ...,
       [0.48235294, 0.43921569, 0.4       , ..., 0.22352941, 0.21568627,
        0.21176471],
       [0.43137255, 0.43137255, 0.42352941, ..., 0.34117647, 0.3372549 ,
        0.3372549 ],
       [0.42352941, 0.42352941, 0.42352941, ..., 0.39215686, 0.4       ,
        0.40392157]])

In [59]:
damsel_tensors = [np.array(damsel[0][0]) for damsel in damsel_train]

In [63]:
damsel_tensors = np.asarray(damsel_tensors)

In [65]:
x_train = damsel_tensors[:17000].reshape(-1, 65536)
x_test = damsel_tensors[17000:].reshape(-1, 65536)

# this is our input placeholder
input_img = Input(shape=(65536,))

# "encoded" is the encoded representation of the input
encoded = Dense(4000, activation='tanh')(input_img)
encoded = Dense(2000, activation='relu')(encoded)
encoded = Dense(1000, activation='relu')(encoded)
encoded = Dense(500, activation='relu')(encoded)
encoded = Dense(10, activation='sigmoid')(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(500, activation='relu')(encoded)
decoded = Dense(1000, activation='relu')(decoded)
decoded = Dense(2000, activation='relu')(decoded)
decoded = Dense(4000, activation='relu')(decoded)
decoded = Dense(1)(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

ValueError: cannot reshape array of size 4352000 into shape (65536)

In [23]:
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

autoencoder.compile(optimizer='SGD', loss='mse')

estop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

train_history = autoencoder.fit(x_train, x_train, epochs=1, batch_size=1, 
                                validation_data=(x_test, x_test), callbacks=[estop])

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 18000 arrays: [array([[0.34509804, 0.35294118, 0.39607843, ..., 0.64313725, 0.65882353,
        0.64705882]]), array([[0.80392157, 0.82352941, 0.82745098, ..., 0.5254902 , 0.49803922,
        0.49411765]]), array([...

In [ ]:
keras.__version__

In [ ]:
tensorflow.__version__

In [ ]:
!pip --upgrade tensorflow==2.0